In [ ]:
import database_faiss as faissdb
import database_sql as db
import openai
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [8]:
model = HuggingFaceEmbeddings(model_name="sentence-transformers/LaBSE")
embedding_dim = len(model.embed_query("hello world"))
import faiss
index = faiss.IndexFlatL2(embedding_dim)
vector_store = faissdb.connect_faiss(embedding_model=model, index=index, faiss_index_dir='../01_data/project_faiss')

C:\Users\ojeem\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
total_vectores = vector_store.index.ntotal
print("Número de vectores indexados:", total_vectores)

Número de vectores indexados: 79044


In [10]:
dimension = vector_store.index.d
print("Dimensionalidad de los vectores:", dimension)

Dimensionalidad de los vectores: 768


In [11]:
query = "¿Cuántas horas de prácticas curriculares equivalen a un crédito ECTS?"

In [12]:
results = vector_store.similarity_search_with_score(query, k=3)
contexto = results[0][0].page_content
id = results[0][0].id
results

[(Document(id='40722', metadata={'page': 4}, page_content='En resumen,  el reconocimiento  de la labor \ndocente (denominada  CREP) para 1 crédito ECTS, \nestará comprendido  entre 11 y 12 horas. De tal \nforma que 1 crédito ECTS contiene 10 horas de \nactividad docente presencial,  y las 2 horas \nrestantes,  como máximo, corresponden  al \nreconocimiento  de actividades.  \n 2. Se mantiene  el reconocimiento  de la labor \ndocente del profesorado  de tal forma\n que se \nasume que 1 crédito ECTS contiene 10 horas de \nactividad docente presencial.'),
  np.float32(0.7500471)),
 (Document(id='16507', metadata={'page': 2}, page_content='titulaciones  oficiales, serán contempladas  en el \nsuplemento  europeo al título (SET) conforme  \ndetermine  la normativa  vigente. \n \nTercero.‐ La redacción  de los apartados  1 y 2 del \nartículo 5 queda de la siguiente manera: \n 1. La duración de las prácticas académicas  externas \nserá la siguiente,  considerando  que 1 crédito ECTS \nequivale

In [ ]:
prompt_cot = (  
    "Sigue este proceso para responder:\n"  
    "1. Analiza la pregunta y el contexto detenidamente.\n"  
    "2. Identifica datos numéricos o términos clave en el contexto.\n"  
    "3. Determina si el contexto contiene información relevante para la pregunta.\n"  
    "4. Si no hay datos suficientes, decláralo explícitamente.\n"  
    "5. Si hay información válida, deriva la respuesta paso a paso.\n\n"  
    "**Pregunta:** {query}\n"  
    "**Contexto UPV:** {context}\n\n"  
    "**Respuesta (formato requerido):**\n"  
    "Razonamiento:\n"  
    "- [Pasos lógicos basados en el contexto]\n\n"  
    "Respuesta final:\n"  
    "- Basado en el contexto: [Sí/No]\n"  
    "- Detalle: [Explicación concisa con datos exactos]"  
)  
def get_response(client, query, context):  
    response = client.chat.completions.create(  
        model="poligpt",  
        messages=[{  
            "role": "user",  
            "content": prompt_cot.format(query=query, context=context)  
        }]  
    )  
    return response.choices[0].message.content

In [19]:
def get_response(client, query, context):
    response = client.chat.completions.create(
        model="poligpt",
        messages = [
            {
                "role": "user",
                "content": f"Responde a la siguiente pregunta usando el contexto suministrado.\n\nPregunta: {query}\nContexto: {context}"
            }
        ]
    )
    return response.choices[0].message.content

In [25]:
client = openai.OpenAI(
        api_key="sk-1C-hWjmHEW05iQjwmr9EnA",
        base_url="https://api.poligpt.upv.es"
    )

In [26]:
answer = get_response(client, query, contexto)
print("Response:", answer)

Response: Razonamiento:
1. **Identificar la pregunta clave**: La pregunta busca saber cuántas horas de prácticas curriculares equivalen a un crédito ECTS.
2. **Analizar el contexto proporcionado**: El contexto se refiere al reconocimiento de la labor docente (CREP) y su relación con los créditos ECTS en la UPV, mencionando que 1 crédito ECTS está relacionado con una cantidad específica de horas de actividad docente presencial y otras actividades.
3. **Extraer datos relevantes**: Según el contexto, 1 crédito ECTS contiene 10 horas de actividad docente presencial, más un máximo de 2 horas por reconocimiento de otras actividades.
4. **Relacionar con la pregunta**: La pregunta sobre prácticas curriculares y su equivalencia en horas a un crédito ECTS se puede relacionar indirectamente con el contexto proporcionado, que habla sobre el tiempo dedicado a la actividad docente presencial.

Respuesta final:
- Basado en el contexto: Sí
- Detalle: Según el contexto de la UPV, 1 crédito ECTS equival

In [ ]:
import textwrap
from typing import List
RAG_PROMPT = textwrap.dedent("""
### Sistema
Eres un asistente que *solo* puede contestar usando la información incluida en el CONTEXTO.

### Instrucciones
1. Si el CONTEXTO está vacío o no contiene datos relevantes para la pregunta, responde exactamente:
   No hay información disponible en el contexto.
2. No inventes ni completes con conocimiento externo.
3. Cuando cites números, nombres propios o fechas, *mantenlos tal cual* aparecen en el CONTEXTO.
4. Responde en español de forma clara y concisa.

### CONTEXTO
{context}

### PREGUNTA
{question}

### RESPUESTA
""").strip()

# ───────────────────────────────────────────
# 1. Plantilla de prompt para respuesta RAG
# ───────────────────────────────────────────
RAG_PROMPT = textwrap.dedent("""
### Sistema
Eres un asistente que *solo* puede contestar usando la información incluida en el CONTEXTO.

### Instrucciones
1. Si el CONTEXTO está vacío o no contiene datos relevantes para la pregunta, responde exactamente:
   No hay información disponible en el contexto.
2. No inventes ni completes con conocimiento externo.
3. Cuando cites números, nombres propios o fechas, *mantenlos tal cual* aparecen en el CONTEXTO.
4. Responde en español de forma clara y concisa.

### CONTEXTO
{context}

### PREGUNTA
{question}

### RESPUESTA
""").strip()

# ───────────────────────────────────────────
# 2. Función de respuesta usando el contexto
# ───────────────────────────────────────────
def answer_from_context(
        client,
        question: str,
        context_chunks: List[str],
        model_name: str = "poligpt"
    ) -> str:
    """
    Devuelve la respuesta a question basándose únicamente en context_chunks.
    - client: Objeto OpenAI o compatible, ya inicializado.
    - model_name: Nombre del modelo que tengas desplegado en tu endpoint.
    """
    # Concatenamos el contexto en un solo string
    context = "\n\n".join(context_chunks).strip()

    # Si no hay contexto devolvemos directamente el fallback, ahorramos una llamada al modelo
    if not context:
        return "No hay información disponible en el contexto."

    prompt = RAG_PROMPT.format(context=context, question=question.strip())

    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content.strip()

# ───────────────────────────────────────────
# 3. Plantilla y función para generar preguntas
# ───────────────────────────────────────────
QUESTION_PROMPT = textwrap.dedent("""
Lee el CONTEXTO y redacta *una sola pregunta* cuya respuesta aparezca explícitamente en él.
No incluyas la respuesta, solo la pregunta.

### CONTEXTO
{context}

### PREGUNTA
""").strip()

def question_from_context(
        client,
        context_chunks: List[str],
        model_name: str = "poligpt"
    ) -> str:
    """
    Genera una pregunta basada exclusivamente en context_chunks.
    Útil para tests de comprensión o para comprobar que el modelo "ve" el contexto.
    """
    context = "\n\n".join(context_chunks).strip()

    # Si no hay contexto tampoco tiene sentido preguntar
    if not context:
        return "No hay información disponible en el contexto."

    prompt = QUESTION_PROMPT.format(context=context)

    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content.strip()